In [1]:
import os
from shapely.geometry import Polygon, Point
import pointpats
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.strtree import STRtree
import numpy as np
from SimuSamp.new_funcs.load.SpatDat import SpatDat
import pandas as pd
from SimuSamp.new_funcs.compute.n_core_sampler import sample_n_cores

In [7]:
parent_filepath = "../../../../Simulated WS Sampling/block_data"
sampleids = ["01_F", "01_G", "02_G", "02_H", "03_B", "03_H", "04_I", "04_N", "05_G", "05_I", "06_H", "07_K", "08_J", "09_E", "10_B", "11_E", "11_F", "12_M", "13_G", "14_I", "15_G", "16_B", "17_I", "18_E", "19_F", "21_I", "22_N", "23_B", "24_I", "25_M", "26_I", "27_H", "28_FSC", "29_L", "30_D"]

microns_per_pixel=0.22715
mm2_per_pixels2 = (microns_per_pixel / 1000) ** 2

In [1]:
# for sampleid in sampleids:
#     # Load sample data
#     spatdat = SpatDat(sampleid, parent_filepath)

#     results_list = []
#     for region in ["tumour", "IM"]:
#         # Perform simulated sampling
#         result = sample_n_cores(spatdat, region=region)

#         results_list.append(result)

#     results = pd.concat(results_list)

#     # Save results per sampleid
#     results.to_excel(f"Data/{sampleid}_sim_results.xlsx", index=False)

#     del(results_list, results)

In [ ]:
combined_df = pd.concat([pd.read_excel(f"Data/{sampleid}_sim_results.xlsx") for sampleid in sampleids])

combined_df.to_excel("simulation_results.xlsx")

In [4]:
combined_df = pd.read_excel("../../../../Data/Simulated/simulation_results.xlsx")

In [16]:
spatdat = SpatDat("01_F", parent_filepath)

In [48]:
cells = spatdat.subset_cells("extended_partition")
cells = cells.drop(columns=["Analysis Region", "CD8", "layer"])
area = spatdat.subset_annotation("extended_partition")
area = gpd.GeoDataFrame(geometry=[area])

In [27]:
small_cores = combined_df[(combined_df["sampleid"] == "01_F") & (combined_df["n_sampled"] == 10) & (combined_df["core_radius"] == 0.3)].iloc[[0], :]

In [29]:
from shapely.wkt import loads

In [35]:
cores = loads(small_cores["cores"].iloc[0])
cores = gpd.GeoDataFrame(geometry=[x for x in cores.geoms])

In [55]:
inter_cells = cells.sjoin(cores, predicate="within")

In [128]:
core_cells_0 = inter_cells[inter_cells["index_right"] == 2].reset_index(drop=True)

In [129]:
core_0 = cores.iloc[[2], :]

In [68]:
distance = 5000 * microns_per_pixel

In [74]:
def edge_correction_weight(point, core, distance):
    # Get fraction of the buffer within the analysis area
    circle = point.buffer(distance)
    intersected_area = circle.intersection(core).area

    weight = circle.area / intersected_area
    
    return weight

In [62]:
from scipy.spatial import KDTree

In [121]:
def compute_nn_per_point(cell_gdf, core, distance=1500 / microns_per_pixel):

    nn_values = []

    tree = KDTree([[p.x, p.y] for p in cell_gdf.geometry])

    for point in cell_gdf.geometry:
        dists, _ = tree.query([point.x, point.y], k=len(cell_gdf), distance_upper_bound=distance)
        # Exclude self-distance
        dists = dists[np.isfinite(dists) & (dists > 0)]
        
        count = np.sum(dists <= distance)

        # Apply edge correction
        weight = edge_correction_weight(point, core, distance)

        nn_values.append(count / weight)
    
    mean_nn = np.mean(nn_values)

    return mean_nn

In [130]:
nn_res = compute_nn_per_point(core_cells_0, core_0)

In [131]:
nn_res

8.919999999999998

In [ ]:
for idx, cores in combined_df.iterrows():
    sampleid = combined_df["sampleid"][idx]

    spatdat = SpatDat(sampleid, parent_filepath)

    cells = spatdat.subset_cells("extended_partition")
    area = spatdat.subset_annotation("extended_partition")
    area = gpd.GeoDataFrame(geometry=[area])

    



    

In [5]:
combined_df

,density_mean,density_stdev,density_sterr,region,sampleid,core_radius,n_cores,n_sampled,iteration,cell_counts_mean,areas_mean,random_density_mean,random_counts_mean,random_counts_stdev,nearest_neighbour_mean,nearest_neighbour_stdev,cores,eligible_points
0,942.295509,0.000000,0.000000,tumour,01_F,0.3,1,1,1,266.0,0.282289,782.884614,221.0,0.000000,15.746936,0.000000,MULTIPOLYGON (((78181.06483821743 68137.667410...,MULTIPOINT (43167.69026380376 68112.7579300690...
1,786.427079,0.000000,0.000000,tumour,01_F,0.3,1,1,2,222.0,0.282289,765.172293,216.0,0.000000,14.396542,0.000000,MULTIPOLYGON (((34561.88749015414 33592.852206...,MULTIPOINT (48685.33261614368 49253.0626731161...
2,1144.215975,0.000000,0.000000,tumour,01_F,0.3,1,1,3,323.0,0.282289,740.375043,209.0,0.000000,13.700271,0.000000,MULTIPOLYGON (((39169.37625920637 31514.587886...,MULTIPOINT (39267.733952468836 33945.994010090...
3,41.545950,0.000000,0.000000,tumour,01_F,0.3,1,1,4,7.0,0.168488,753.762233,127.0,0.000000,100.532737,0.000000,MULTIPOLYGON (((112404.27064424715 74953.18518...,MULTIPOINT (86020.0605375199 72232.22705637143...
4,474.690219,0.000000,0.000000,tumour,01_F,0.3,1,1,5,134.0,0.282289,775.799686,219.0,0.000000,17.184455,0.000000,MULTIPOLYGON (((58697.65632641347 44078.114221...,MULTIPOINT (49619.9028111772 65301.52862024358...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104995,605.529380,300.829066,95.130504,IM,30_D,1.0,10,10,46,1579.3,2.564723,498.085497,1303.9,416.523097,16.786816,3.401143,MULTIPOLYGON (((68648.9545890411 28052.2738620...,MULTIPOINT (46082.30176591623 40432.4134998454...
104996,608.358651,390.529125,123.496153,IM,30_D,1.0,10,10,47,1573.9,2.493670,494.598282,1249.7,382.951185,17.511877,3.775624,MULTIPOLYGON (((59808.46071092194 61838.123107...,MULTIPOINT (46581.04080999675 58896.4761611910...
104997,582.406791,425.887383,134.677416,IM,30_D,1.0,10,10,48,1343.9,2.343723,493.272677,1163.6,344.316192,19.598137,6.660760,MULTIPOLYGON (((70791.86438568051 50694.344318...,MULTIPOINT (54917.21951858724 38363.8395503154...
104998,500.361800,286.405464,90.569360,IM,30_D,1.0,10,10,49,1302.7,2.586026,511.996391,1335.8,356.421324,16.973485,2.865569,MULTIPOLYGON (((56332.85011476228 62255.528745...,MULTIPOINT (63399.106349011876 32418.399414559...
